<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">

# AST4310 2023, Project 1

</div>

This notebook contains additional background information and/or tips for each of the exercises. Some general notes:

* Read the questions carefully and try to answer all points. It often happens that students miss easy points because they did not follow what was spelled out in the questions.
* Please do not edit the green cells, and definitely do not split the questions
* Do your best to answer each exercise in one coherent narrative, meaning that you should try to frame the questions in a contiguous body using text, figures, and code (when appropriate), using your own words, instead of just mechanically answering the questions.  
* When possible, demonstrate your findings via code, plots, etc. It is better to have the code output the results than writing the results in a markdown cell. This way, if something in the code or data changes, it is easier to update. For even nicer visual impact, it is possible to format the output of code calls and even use HTML (Jupyter makes it easy to display HTML/Markdown).
* Try and keep your answers concise and to the point. What we are looking for is your physical insight, not your ability to write long essays. Good scientific writing is often to the point, but it takes a while to learn the difference between short and concise, and how much should you write. 

### Exercise 1: Line formation in optically thick medium

In this exercise, you will need to create arrays for extinction as function of frequency and source function as function of height (or depth). Assume that the extinction is the same at all heights. You need to set up an array for height, and then integrate the extinction to obtain the optical depth. Once you have the optical depth and source function, you can obtain the intensity by using the formal solution to the transport equation.

### Exercise 2: The Boltzmann and Saha laws

The Boltzmann law determines the excitation of atomic species in equilibrium, while the Saha law determines the ionisation. In this exercise, we recommend you use the provided hydrogen model atom in the file `H_atom.txt`, and make use of the `Atom` class that we worked on during the classes. 

### Exercise 3: Magnesium

#### NIST Atomic Levels form

Building a model atom by hand using the NIST level energies can be very time demanding. Fortunately, it is possible to automate the process to some extent. By changing the output format of the NIST form, it is possible to download the results into a file and and process it. Only one atomic stage (e.g. He II) can be downloaded at a time. 

We include below a simple python function called `read_NIST_species()` that can read the NIST level energies, statistical weights, and ionisation potential for the output of a NIST search, when a tab-delimited output is chosen. To obtain a file format that this function understands, you need to do a NIST search with the following parameters:

- Level units cm-1
- Format output: Tab-delimited
- Display output: in its entirety
- Energy ordered
- Level information: only select "Principal term",  "Level" and "g"
- No bibliographic references nor level splitting

Any deviation from these will likely cause the code to fail.

In [2]:
def read_NIST_species(input_file):
    """
    Reads data from a text file downloaded from NIST database with
    level energies (4th column and statistical weights (2nd column).
    """
    data = open(input_file, 'r').readlines()[1:]  # skip header
    nlevels = len(data)
    g = numpy.zeros(nlevels, dtype='i') - 1
    chi = numpy.zeros(nlevels, dtype='f') - 1
    chi_ion = 0.
    
    for i, line in enumerate(data):
        entries = line.split('\t')
        if entries[1] != '':
            g[i] = int(entries[1])
            chi[i] = float(entries[3].strip('"').strip('[').strip(']'))
        if entries[0].strip('"').lower() == "limit":
            chi_ion = float(entries[3].strip('"').strip('[').strip(']'))
            break
    # clean up missing values
    mask = (g >= 0) & (chi >= 0)
    return g[mask], chi[mask], chi_ion

Remember that the NIST levels of a given stage are relative to the ground level energy of that stage. You need to take this into consideration when building a model atom that includes several stages.

#### Collisional Ionisation Equilibrium

The Collisional Ionisation Equilibrium (CIE) approximation is valid in low-density plasmas, while the Saha law is valid in thermodynamical equilibrium. Unlike Saha, the CIE is *not* in detailed balance. In low-density plasmas, collisions between atoms, or between atoms and electrons are rare, which means that if an atom is collisionaly excited or ionised, it is much more likely to de-excite or recombine radiatively (and not collisionally). The CIE in essence means that all ionisation happens collisionally (no photons involved) and all recombination happens radiatively (photon creation). 

You can read more detail about the CIE in chapter 5 of the book [Astrophysics of the Diffuse Universe](https://link.springer.com/book/10.1007/978-3-662-05866-4) by Mike Dopita and Ralph Sutherland (from the UiO network, you can download a pdf version for free)